In [1]:
import cv2
import numpy as np

# Διαβάζουμε την εικόνα
image = cv2.imread('leaf.jpg')

# Μετατροπή σε κλίμακα του γκρι
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Κατωφλίωση
threshold_value = 220
_, binary_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

# Εμφάνιση των εικόνων
cv2.imshow('Grayscale Image', gray_image)
cv2.imshow('Binary Image', binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
 

In [2]:
# Find contours using the Moore boundary tracing algorithm
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Select the largest contour
largest_contour = max(contours, key=cv2.contourArea)

# Create a white background image
contour_image = np.ones_like(binary_image) * 255

# Draw the contour with green color (0, 255, 0) on the white background image
cv2.drawContours(contour_image, [largest_contour], -1, (0, 255, 0), thickness=2)

# Display the result
cv2.imshow('Contour Image', contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
# Get the bounding rectangle of the contour
x, y, w, h = cv2.boundingRect(largest_contour)

# Create a new image with the dimensions of the bounding rectangle
contour_image = np.ones((h, w, 3), dtype=np.uint8) * 255

# Adjust the contour points so that they fit within the new image
adjusted_contour = largest_contour - np.array([x, y])

# Draw the contour with green color (0, 255, 0) on the new image
cv2.drawContours(contour_image, [adjusted_contour], -1, (0, 255, 0), thickness=2)

# Display the result
cv2.imshow('Contour Image', contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
# Υπολογισμός των περιγραφέων Fourier
adjusted_contour_float32 = adjusted_contour.astype(np.float32)
fourier_descriptors = cv2.dft(adjusted_contour_float32, flags=cv2.DFT_COMPLEX_OUTPUT)
print(fourier_descriptors)

[[[1078.  718.]]

 [[-180.  180.]]

 [[   0.    0.]]

 [[-898. -898.]]]


In [8]:
# Αρχικά, υπολογίζουμε τον αριθμό των περιγραφέων Fourier
num_descriptors = fourier_descriptors.shape[0]

# Ορίζουμε τα ποσοστά που θέλουμε να χρησιμοποιήσουμε
percentages = [100, 50, 10, 1]

# Loop through the percentages
for percent in percentages:
    # Calculate the number of descriptors to use based on the percentage
    num_descriptors_to_use = int(percent / 100 * num_descriptors)

    # Select the first 'num_descriptors_to_use' Fourier descriptors
    selected_descriptors = fourier_descriptors[:num_descriptors_to_use]

    # Inverse Fourier transform to reconstruct the contour
    reconstructed_contour = cv2.idft(selected_descriptors, flags=cv2.DFT_COMPLEX_INPUT | cv2.DFT_REAL_OUTPUT)

    # Extract the real part of the reconstructed contour
    reconstructed_contour = np.abs(reconstructed_contour)

    # Normalize the reconstructed contour to the range [0, 255]
    reconstructed_contour = cv2.normalize(reconstructed_contour, None, 0, 255, cv2.NORM_MINMAX)

    # Convert the reconstructed contour to 8-bit unsigned integer
    reconstructed_contour = reconstructed_contour.astype(np.uint8)

    # Print the reconstructed contour and the number of non-zero elements
    print(f"Reconstructed Contour {percent}%:")
    print(reconstructed_contour)
    print(f"Number of non-zero elements: {np.count_nonzero(reconstructed_contour)}")

    # Check if the reconstructed contour contains any valid points
    if np.count_nonzero(reconstructed_contour) > 0:
        # Create a white background image
        contour_image = np.ones_like(reconstructed_contour) * 255

        # Draw the reconstructed contour in black
        cv2.drawContours(contour_image, [reconstructed_contour], -1, (0, 0, 0), thickness=2)

        # Display the reconstructed contour
        cv2.imshow(f'Reconstructed Contour {percent}%', contour_image)
        cv2.waitKey(0)
    else:
        print(f"No valid contour points found for {percent}%")

cv2.destroyAllWindows()



Reconstructed Contour 100%:
[[  0]
 [  0]
 [255]
 [255]]
Number of non-zero elements: 2


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\drawing.cpp:2538: error: (-215:Assertion failed) npoints > 0 in function 'cv::drawContours'
